#### DenseNet on CIFAR 10

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


In [2]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout
from tensorflow.keras.layers import concatenate, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
import math

In [3]:
batch_size = 64
epochs = 120
data_augmentation = True

In [4]:
num_classes = 10
num_dense_blocks = 3
use_max_pool = False

In [5]:
growth_rate = 12
depth = 100
num_bottleneck_layers = (depth-4)//(2*num_dense_blocks)

In [6]:
num_filters_bef_dense_block = 2 * growth_rate
compression_factor = 0.5

In [7]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [8]:
input_shape = x_train.shape[1:]

In [9]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print("x_train shape: ",x_train.shape)
print('y_train shape: ',y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape:  (50000, 32, 32, 3)
y_train shape:  (50000, 1)
50000 train samples
10000 test samples


In [10]:
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [11]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 100:
        lr *= 0.5e-3
    elif epoch > 95:
        lr *= 1e-3
    elif epoch > 85:
        lr *= 1e-2
    elif epoch > 70:
        lr *= 1e-1
    print('Learning Rate: ',lr)
    return lr

In [12]:
inputs = Input(shape=input_shape)
x = BatchNormalization()(inputs)
x = Activation('relu')(x)
x = Conv2D(num_filters_bef_dense_block,
          kernel_size=3,
          padding='same',
          kernel_initializer='he_normal')(x)
x = concatenate([inputs,x])

In [13]:
for i in range(num_dense_blocks):
    for j in range(num_bottleneck_layers):
        y = BatchNormalization()(x)
        y = Activation('relu')(y)
        y = Conv2D(4*growth_rate,
                  kernel_size=1,
                  padding='same',
                  kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        x = concatenate([x,y])
    
    if i == num_dense_blocks - 1:
        continue
    
    num_filters_bef_dense_block += num_bottleneck_layers * growth_rate
    num_filters_bef_dense_block = int(num_filters_bef_dense_block) * compression_factor
    y = BatchNormalization()(x)
    y = Conv2D(num_filters_bef_dense_block,
              kernel_size=1,
              padding='same',
              kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)

x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)
outputs = Dense(num_classes,
               kernel_initializer='he_normal',
               activation='softmax')(y)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(1e-3),
             metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 3)    0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   672         activation[0][0]                 
______________________________________________________________________________________________

In [14]:
save_dir = os.path.join(os.getcwd(),'desnenet_cifar10_saved_models')
model_name = 'cifar10_densenet_model.{epoch:02d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir,model_name)

In [15]:
checkpoint = ModelCheckpoint(filepath=filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                              cooldown=0,
                              patience=5,
                              min_lr=0.5e-6)
callbacks = [checkpoint, lr_scheduler,lr_scheduler ]

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train,y_train,
             batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test,y_test),
              shuffle=True,
              callbacks=callbacks)

else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=False)
    datagen.fit(x_train)
    step_per_epoch = math.ceil(len(x_train)/batch_size)
    model.fit(x=datagen.flow(x_train,y_train,batch_size=batch_size),
             epochs=epochs,
             validation_data=(x_test,y_test),
             steps_per_epoch=step_per_epoch,
             callbacks=callbacks)

Using real-time data augmentation.
Epoch 1/120
Learning Rate:  0.001
Learning Rate:  0.001
  3/782 [..............................] - ETA: 7:36:03 - loss: 4.2197 - acc: 0.0773

In [ ]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accurcay: ')